In [1]:
!pip install -qU pinecone-client

In [5]:
import pinecone
pinecone.init(
    api_key='ba94c143-b7f7-42bd-bbb7-4261f136dd55',
    environment='us-west1-gcp-free'
)
pinecone.list_indexes()

C:\Users\yasshah9\AppData\Roaming\Python\Python311\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


['it-threats']

In [3]:
!pip install -qU pip python-dateutil keras==2.4.0 scikit-learn matplotlib==3.1.0 seaborn

ERROR: Cannot install keras==2.4.0 and tensorflow because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [2]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
from tensorflow import keras
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [3]:
# Pick a name for the new service
index_name = 'it-threats'

In [ ]:
# Make sure service with the same name does not exist
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

In [ ]:
pinecone.create_index(name=index_name, dimension=128, metric='cosine')

In [6]:
index = pinecone.Index(index_name=index_name)

In [11]:
data = pd.read_csv('./UNSW_NB15_training-set.csv')
data.attack_cat.value_counts()

attack_cat
Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
Name: count, dtype: int64

In [11]:
test = pd.read_csv('./UNSW_NB15_testing-set.csv')
test.attack_cat.value_counts()

attack_cat
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: count, dtype: int64

In [12]:
data1=data.drop(['proto','service','state','attack_cat'],axis=1)

In [13]:
data1

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,1,0.000011,2,0,496,0,90909.090200,254,0,1.803636e+08,...,1,1,2,0,0,0,1,2,0,0
1,2,0.000008,2,0,1762,0,125000.000300,254,0,8.810000e+08,...,1,1,2,0,0,0,1,2,0,0
2,3,0.000005,2,0,1068,0,200000.005100,254,0,8.544000e+08,...,1,1,3,0,0,0,1,3,0,0
3,4,0.000006,2,0,900,0,166666.660800,254,0,6.000000e+08,...,2,1,3,0,0,0,2,3,0,0
4,5,0.000010,2,0,2126,0,100000.002500,254,0,8.504000e+08,...,2,1,3,0,0,0,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,2,0,104,0,200000.005100,254,0,8.320000e+07,...,1,1,2,0,0,0,2,1,0,0
82328,82329,1.106101,20,8,18062,354,24.410067,254,252,1.241044e+05,...,1,1,1,0,0,0,3,2,0,0
82329,82330,0.000000,1,0,46,0,0.000000,0,0,0.000000e+00,...,1,1,1,0,0,0,1,1,1,0
82330,82331,0.000000,1,0,46,0,0.000000,0,0,0.000000e+00,...,1,1,1,0,0,0,1,1,1,0


In [12]:
test_data=test.drop(['proto','service','state','attack_cat'],axis=1)

In [15]:
X = data1.iloc[:,:-1]
y = data1.iloc[:,-1]

In [16]:
len(test_data)

175341

In [17]:
X_test = test_data.iloc[:,:-1]
y_test = test_data.iloc[:,-1]

In [18]:
# model = tf.keras.Sequential([
#   tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(40,)),  # input shape required
#   tf.keras.layers.Dense(64, activation=tf.nn.relu),
#   tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
# ])
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128,
                                input_dim=X.shape[1],
                                activation=tf.nn.relu,
                                kernel_initializer='he_normal')
)
model.add(tf.keras.layers.Dense(64,
                                activation=tf.nn.relu,
                                kernel_initializer='he_normal')
)
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5248      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13,569
Trainable params: 13,569
Non-trainable params: 0
_________________________________________________________________


In [20]:
history=model.fit(
    X,y,
    batch_size=64,
    epochs=20,
    shuffle=True,
    verbose=2,
    validation_data=(X_test,y_test)
)
_, accuracy = model.evaluate(X_test,y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))
model.summary()

Epoch 1/20
1287/1287 - 5s - loss: 4660720.5000 - accuracy: 0.6469 - val_loss: 1531718.7500 - val_accuracy: 0.7847 - 5s/epoch - 4ms/step
Epoch 2/20
1287/1287 - 4s - loss: 2430431.2500 - accuracy: 0.6605 - val_loss: 3484347.7500 - val_accuracy: 0.7828 - 4s/epoch - 3ms/step
Epoch 3/20
1287/1287 - 4s - loss: 2164912.5000 - accuracy: 0.6791 - val_loss: 1283802.2500 - val_accuracy: 0.3853 - 4s/epoch - 3ms/step
Epoch 4/20
1287/1287 - 4s - loss: 1790975.6250 - accuracy: 0.6793 - val_loss: 2872363.2500 - val_accuracy: 0.3485 - 4s/epoch - 3ms/step
Epoch 5/20
1287/1287 - 4s - loss: 1425690.1250 - accuracy: 0.6894 - val_loss: 686840.4375 - val_accuracy: 0.6769 - 4s/epoch - 3ms/step
Epoch 6/20
1287/1287 - 4s - loss: 1423460.7500 - accuracy: 0.6853 - val_loss: 244056.6250 - val_accuracy: 0.7975 - 4s/epoch - 3ms/step
Epoch 7/20
1287/1287 - 4s - loss: 1111666.1250 - accuracy: 0.6855 - val_loss: 475905.8438 - val_accuracy: 0.6729 - 4s/epoch - 3ms/step
Epoch 8/20
1287/1287 - 4s - loss: 990865.8125 - acc

In [1]:
model.save("my_model")

NameError: name 'model' is not defined

In [7]:
my_model = keras.models.load_model("my_model")

In [8]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5248      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13,569
Trainable params: 13,569
Non-trainable params: 0
_________________________________________________________________


In [9]:
my_model.save(r"C:\Users\yasshah9\Desktop\Threat Detection\backend\model")

INFO:tensorflow:Assets written to: C:\Users\yasshah9\Desktop\Threat Detection\backend\model\assets


INFO:tensorflow:Assets written to: C:\Users\yasshah9\Desktop\Threat Detection\backend\model\assets


In [10]:
layer_name = 'dense' 
intermediate_layer_model = Model(inputs=my_model.input,
                                 outputs=my_model.get_layer(layer_name).output)
intermediate_layer_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_input (InputLayer)    [(None, 40)]              0         
                                                                 
 dense (Dense)               (None, 128)               5248      
                                                                 
Total params: 5,248
Trainable params: 5,248
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tqdm import tqdm
items_to_upload = []

model_res = intermediate_layer_model.predict(K.constant(data1.iloc[:,:-1]))

for i, res in tqdm(zip(data.iterrows(), model_res), total=len(model_res)):
    benign_or_attack = i[1]['attack_cat'][:3]
    items_to_upload.append((benign_or_attack + '_' + str(i[0]), res.tolist()))

In [ ]:
import numpy as np
np.shape(items_to_upload)

In [ ]:
import itertools

def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

In [ ]:
NUMBER_OF_ITEMS = len(items_to_upload)

for batch in chunks(items_to_upload[:NUMBER_OF_ITEMS], 50):
    index.upsert(vectors=batch)

In [ ]:
items_to_upload.clear()

In [28]:
index.describe_index_stats()

{'dimension': 128,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 82332}},
 'total_vector_count': 82332}

In [13]:
test_data1=test.drop(['proto','service','state'],axis=1)

In [14]:
data_sample1 = test_data1.iloc[50000:50001,:]
# data_sample1.attack_cat.value_counts()
data_sample1 = pd.concat([data_sample1, pd.DataFrame(data_sample1)], ignore_index=True)
print(data_sample1)

    id        dur  spkts  dpkts  sbytes  dbytes      rate  sttl  dttl  \
0  101  33.787666     14     18    1684   10168  0.917495    31    29   
1  101  33.787666     14     18    1684   10168  0.917495    31    29   

        sload  ...  ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  \
0  370.312653  ...                 1               2             0   
1  370.312653  ...                 1               2             0   

   ct_ftp_cmd  ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  is_sm_ips_ports  \
0           0                 1           4           5                0   
1           0                 1           4           5                0   

   attack_cat  label  
0      Normal      0  
1      Normal      0  

[2 rows x 42 columns]


In [15]:
data_sample=data_sample1.drop(['attack_cat'],axis=1)

In [16]:
from tqdm import tqdm

In [17]:
y_true = []
y_pred = []

BATCH_SIZE = 1

for i in tqdm(range(0, len(data_sample), BATCH_SIZE)):
    test_data = data_sample.iloc[i:i+BATCH_SIZE, :]
    data_sample2=data_sample1.iloc[i:i+BATCH_SIZE, :]
    # Create vector embedding using the model
    test_vector = intermediate_layer_model.predict(K.constant(test_data.iloc[:, :-1]))
    # Query using the vector embedding
    query_results = []

    for xq in test_vector.tolist():
        query_res = index.query(xq, top_k=20)
        query_results.append(query_res)
    
    ids = [res.id for result in query_results for res in result.matches]
    
    for label, res in zip(data_sample2.attack_cat.values, query_results):
        # Add to the true list
        if label == 'Normal':
            y_true.append(0)
        else:
            y_true.append(1)
        
        counter = Counter(match.id.split('_')[0] for match in res.matches)

        # Add to the predicted list
        if counter['Nor']:
            y_pred.append(0)
        else:
            y_pred.append(1)

  0%|          | 0/2 [00:00<?, ?it/s]

1/1 [==============================] - 0s 104ms/step


 50%|█████     | 1/2 [00:01<00:01,  1.15s/it]

1/1 [==============================] - 0s 25ms/step


100%|██████████| 2/2 [00:01<00:00,  1.24it/s]


In [18]:
# Calculate accuracy
acc = accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print(f"Accuracy: {acc:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")

Accuracy: 0.000
Precision: 0.000
Recall: 0.000


C:\Users\yasshah9\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
